In [1]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *
from functions_multiple_vers import *

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5



In [4]:
random_sequences_collection[1]

['gcccta',
 'tcgctg',
 'acccact',
 'tattaatac',
 'caaaagata',
 'gtcagc',
 'gcaggg',
 'gatgttt',
 'gtccacta',
 'agaatt']

In [3]:
random_sequences_collection=[]
for i in range(0,100):
    random_pile=[]
    for j in range (0,10):
        a_sequence=[]
        for k in range(0,random.randrange(5,10)):
            a_sequence.append(random.choice(['a','c','t','g']))
        a_sequence=''.join(a_sequence)
        random_pile.append(a_sequence)

    random_sequences_collection.append(random_pile)

print(random_sequences_collection)

[['tttggta', 'tggtccat', 'caattatc', 'aagcctttg', 'ggaccgta', 'gactggg', 'gacttgta', 'cgggg', 'cttttatag', 'gttcatg'], ['gcccta', 'tcgctg', 'acccact', 'tattaatac', 'caaaagata', 'gtcagc', 'gcaggg', 'gatgttt', 'gtccacta', 'agaatt'], ['agagg', 'gctggccc', 'ctcgcgcgt', 'tttctc', 'gcgtcatag', 'tcttgttg', 'gagcta', 'tggcgc', 'gcgga', 'tgatg'], ['cgtgct', 'tcaat', 'gactt', 'ccagttgcc', 'tcgttgaat', 'atcccaa', 'tgctcgtag', 'gggtgac', 'tagcgt', 'gcgaaa'], ['cacgt', 'gacgcg', 'aagggtc', 'acgcgaacg', 'gagagac', 'acaaac', 'taccat', 'tacatt', 'tttca', 'gctcaatat'], ['tcgctg', 'accaggc', 'caatctc', 'gcagagg', 'cgccgtag', 'gaggctct', 'agtcatagt', 'acacatgg', 'tactgac', 'agtccgg'], ['agcctgg', 'acgacgcc', 'atctggc', 'tcatgcg', 'cacagagcg', 'cgtagc', 'gccatct', 'tccccgata', 'aggtgt', 'ccaggccac'], ['ccgcttcat', 'ctctcacc', 'caagcgag', 'ggggtacgc', 'gcccttc', 'tttcct', 'cgtgc', 'tatttct', 'tcatgca', 'ttatccga'], ['ttgtgg', 'agtttatct', 'tatgtc', 'ggtttcgcc', 'acataagta', 'agctgc', 'aaaaaaact', 'gcctggg'

In [5]:
costs_new=[]
costs_old=[]
for random_pile in random_sequences_collection:
    cost=new_assembly(random_pile,score_matrix,gap_cost)[4]
    costs_new.append(cost)
for random_pile in random_sequences_collection:
    cost=sp_approx(random_pile,score_matrix,gap_cost)[0]
    costs_old.append(cost)
print(costs_new)
print(costs_old)
costs_new=np.array(costs_new)
costs_old=np.array(costs_old)
differences=np.subtract(costs_new,costs_old)
print(differences)

yes; i was updated
Here comes the distance matrix produced by the alignments: 

[[ 0. 24. 24. 31. 22. 19. 17. 19. 17. 14.]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
Here comes the distance matrix produced by the alignments: 

[[ 0. 24. 24. 31. 22. 19. 17. 19. 17. 14.]
 [24.  0. 18. 21. 17. 23. 20. 24. 26. 17.]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan

SystemExit: 

/home/ana/mambaforge/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
count=np.count_nonzero(differences<0)
print("Your algo was better than the classic in "+ str(count)+"% of the time")

NameError: name 'differences' is not defined